In [1]:
!rm -r /kaggle/working/*
%cd /kaggle/working

rm: cannot remove '/kaggle/working/*': No such file or directory
/kaggle/working


In [2]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [3]:
import yaml
from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base

MODEL_NAME = "xgboost"
CFG = yaml.safe_load(open(os.path.join(PACKAGE_DIR, "config.yaml"), "r"))
print(CFG[MODEL_NAME]["execution"]["exp_id"])
CFG["output_dir"] = f"/kaggle/output/{CFG[MODEL_NAME]['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

init_logger(f"{ CFG[MODEL_NAME]['execution']['exp_id']}.log")
logger = get_logger("main")
seed_base(CFG[MODEL_NAME]["execution"]["seed"])

2024-11-16 09:27:04.641922: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 09:27:04.670105: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


exp_005


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
set seed: 46


In [4]:
import warnings

import numpy as np
import polars as pl

warnings.filterwarnings("ignore")

In [5]:
train = pl.read_csv(os.path.join(CFG["dataset"]["competition_dir"], "train_features.csv"))
train = (
    train.with_columns(
        pl.col("ID").str.split_exact("_", n=1).struct.rename_fields(["sceneID", "offset"]).alias("fields")
    )
    .unnest("fields")
    .with_columns(pl.col("offset").cast(pl.Float32))
)
print(train.shape)
train.head(1)

(43371, 32)


ID,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,leftBlinker,rightBlinker,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,sceneID,offset
str,f64,f64,f64,f64,f64,bool,f64,bool,str,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f32
"""00066be8e20318869c38c66be46663…",5.701526,1.538456,-2.165777,-139.0,0.0,false,0.25,true,"""drive""",false,false,2.82959,0.032226,0.045187,6.231999,0.065895,0.107974,9.785009,0.124972,0.203649,13.485472,0.163448,0.302818,17.574227,0.174289,0.406331,21.951269,0.199503,0.485079,"""00066be8e20318869c38c66be46663…",320.0


## 特徴量生成


In [6]:
from penguinml.utils.contena import FeatureContena

features = FeatureContena()
basic_features = [
    "vEgo",
    "aEgo",
    "steeringAngleDeg",
    "steeringTorque",
    "brake",
    "brakePressed",
    "gas",
    "gasPressed",
    "leftBlinker",
    "rightBlinker",
    "offset",
]
for c in basic_features:
    train = train.with_columns(pl.col(c).cast(pl.Float32))
features.add_num_features(basic_features)
features.add_cat_features(["gearShifter"])

In [ ]:
# 前後のフレームとのラグ特徴量
train = train.sort(["sceneID", "offset"])
for c in basic_features:
    for diff in [-1, 1]:
        train = train.with_columns(
            pl.col(c).diff(n=diff).over("sceneID").alias(f"{c}_diff_{diff}"),
            pl.col(c).diff(n=diff).over("sceneID").alias(f"{c}_shift_{diff}"),
        )
        features.add_num_features([f"{c}_diff_{diff}", f"{c}_shift_{diff}"])

In [8]:
# train = train.with_columns(
#     (pl.col("vEgo") / pl.col("aEgo")).alias("vEgo/aEgo"),
# )
# features.add_num_features(["vEgo/aEgo"])

## ターゲット列を分解


In [9]:
from const import TARGET_COLS

train = (
    train.unpivot(index="ID", on=TARGET_COLS, variable_name="target_name", value_name="target")
    .join(
        train.drop(TARGET_COLS),
        on="ID",
        how="left",
    )
    .with_columns(
        (
            pl.col("target_name").map_elements(lambda x: float(x.split("_")[1]), return_dtype=pl.Float32) * 0.5 + 0.5
        ).alias("dt"),
        pl.col("target_name").map_elements(lambda x: x.split("_")[0], return_dtype=str).alias("xyz"),
    )
    .with_columns(
        pl.col("target_name").alias("target_name_original"),
    )
)
features.add_cat_features(["target_name", "xyz"])
train.head(1)

ID,target_name,target,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,leftBlinker,rightBlinker,sceneID,offset,vEgo_diff_-1,vEgo_shift_-1,vEgo_diff_1,vEgo_shift_1,aEgo_diff_-1,aEgo_shift_-1,aEgo_diff_1,aEgo_shift_1,steeringAngleDeg_diff_-1,steeringAngleDeg_shift_-1,steeringAngleDeg_diff_1,steeringAngleDeg_shift_1,steeringTorque_diff_-1,steeringTorque_shift_-1,steeringTorque_diff_1,steeringTorque_shift_1,brake_diff_-1,brake_shift_-1,brake_diff_1,brake_shift_1,brakePressed_diff_-1,brakePressed_shift_-1,brakePressed_diff_1,brakePressed_shift_1,gas_diff_-1,gas_shift_-1,gas_diff_1,gas_shift_1,gasPressed_diff_-1,gasPressed_shift_-1,gasPressed_diff_1,gasPressed_shift_1,leftBlinker_diff_-1,leftBlinker_shift_-1,leftBlinker_diff_1,leftBlinker_shift_1,rightBlinker_diff_-1,rightBlinker_shift_-1,rightBlinker_diff_1,rightBlinker_shift_1,offset_diff_-1,offset_shift_-1,offset_diff_1,offset_shift_1,dt,xyz,target_name_original
str,str,f64,f32,f32,f32,f32,f32,f32,f32,f32,str,f32,f32,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str
"""00066be8e20318869c38c66be46663…","""x_0""",2.82959,5.701526,1.538456,-2.165777,-139.0,0.0,0.0,0.25,1.0,"""drive""",0.0,0.0,"""00066be8e20318869c38c66be46663…",320.0,-5.474767,-5.474767,null,null,1.258575,1.258575,null,null,9.45992,9.45992,null,null,-95.0,-95.0,null,null,0.0,0.0,null,null,0.0,0.0,null,null,0.25,0.25,null,null,1.0,1.0,null,null,0.0,0.0,null,null,-1.0,-1.0,null,null,-100.0,-100.0,null,null,0.5,"""x""","""x_0"""


In [10]:
# dt秒後の特徴
train = train.with_columns(
    # vt
    (pl.col("vEgo") * pl.col("dt").cast(pl.Float32)).alias("linear_movement@dt"),
    # vt + 0.5at^2
    ((pl.col("vEgo") + 0.5 * pl.col("aEgo") * pl.col("dt").cast(pl.Float32) ** 2).alias("movement@dt")),
    # v + at
    (pl.col("vEgo") + pl.col("aEgo") * pl.col("dt").cast(pl.Float32)).alias("velocity@dt"),
    # # cos
    # (pl.col("steeringAngleDeg").map_elements(lambda x: np.cos(np.deg2rad(x)), return_dtype=pl.Float32)).alias("cos"),
    # # sin
    # (pl.col("steeringAngleDeg").map_elements(lambda x: np.sin(np.deg2rad(x)), return_dtype=pl.Float32)).alias("sin"),
).with_columns(
    # (pl.col("movement@dt") * pl.col("cos")).alias("movement@dt*cos"),
    # (pl.col("movement@dt") * pl.col("sin")).alias("movement@dt*sin"),
    # (pl.col("linear_movement@dt") * pl.col("cos")).alias("linear_movement@dt*cos"),
    # (pl.col("linear_movement@dt") * pl.col("sin")).alias("linear_movement@dt*sin"),
    # (pl.col("velocity@dt") * pl.col("cos")).alias("velocity@dt*cos"),
    # (pl.col("velocity@dt") * pl.col("sin")).alias("velocity@dt*sin"),
)

features.add_num_features(
    [
        "linear_movement@dt",
        "movement@dt",
        "velocity@dt",
        # "cos",
        # "sin",
        # "movement@dt*cos",
        # "movement@dt*sin",
        # "linear_movement@dt*cos",
        # "linear_movement@dt*sin",
        # "velocity@dt*cos",
        # "velocity@dt*sin",
    ]
)

In [11]:
# シーン内の集約特徴量
for c in features.num_features():
    train = train.with_columns(
        pl.col(c).mean().over("sceneID").alias(f"{c}_mean"),
        pl.col(c).std().over("sceneID").alias(f"{c}_std"),
        pl.col(c).max().over("sceneID").alias(f"{c}_max"),
        pl.col(c).min().over("sceneID").alias(f"{c}_min"),
    )
    features.add_num_features([f"{c}_mean", f"{c}_std", f"{c}_max", f"{c}_min"])

In [12]:
from const import CATEGORY_MAPPING

for c in features.num_features():
    train = train.with_columns(pl.col(c).cast(pl.Float32))

for c in features.cat_features():
    mapping = CATEGORY_MAPPING[c]
    train = train.with_columns(pl.col(c).replace_strict(mapping).cast(pl.Int32))

## CV Split


In [13]:
train_folds = pl.read_csv(CFG["dataset"]["train_fold_path"])
train = train.join(train_folds, on="sceneID", how="left")
assert train["fold"].null_count() == 0

## Training


In [14]:
from penguinml.gbdt.xgboost import fit_xgb, inference_xgb
from tqdm import tqdm

dfs = []
for c in tqdm(sorted(TARGET_COLS, key=lambda x: x.split("_")[1], reverse=True)):
    print("==" * 10 + c + "==" * 10)
    this_df = train.filter(pl.col("target_name_original") == c)
    oof, models = fit_xgb(
        data=this_df,
        features=features,
        params=CFG[MODEL_NAME]["params"],
        target_col="target",
        fold_col="fold",
        target_type="regression",
        verbose=50000,
    )
    this_df = this_df.with_columns(pl.Series("oof", oof))
    dfs.append(this_df)
train = pl.concat(dfs)

  0%|          | 0/18 [00:00<?, ?it/s]

====================x_5====================
== fold 0 ==
[0]	validation_0-mae:17.66778
[1895]	validation_0-mae:0.70401
== fold 1 ==
[0]	validation_0-mae:17.69517
[5535]	validation_0-mae:0.73028
== fold 2 ==
[0]	validation_0-mae:17.90453
[2608]	validation_0-mae:0.71290
== fold 3 ==
[0]	validation_0-mae:17.77304
[3810]	validation_0-mae:0.70447
== fold 4 ==
[0]	validation_0-mae:17.38742
[2172]	validation_0-mae:0.71601


  6%|▌         | 1/18 [01:42<29:05, 102.66s/it]

====================y_5====================
== fold 0 ==
[0]	validation_0-mae:0.99735
[2055]	validation_0-mae:0.46758
== fold 1 ==
[0]	validation_0-mae:1.07295
[1731]	validation_0-mae:0.48668
== fold 2 ==
[0]	validation_0-mae:1.06187
[1978]	validation_0-mae:0.48810
== fold 3 ==
[0]	validation_0-mae:1.00575
[2095]	validation_0-mae:0.48441
== fold 4 ==
[0]	validation_0-mae:1.09593
[2027]	validation_0-mae:0.50476


 11%|█         | 2/18 [02:45<21:09, 79.35s/it] 

...

 89%|████████▉ | 16/18 [14:28<01:32, 46.10s/it]

====================y_0====================
== fold 0 ==
[0]	validation_0-mae:0.05734
[1563]	validation_0-mae:0.03171
== fold 1 ==
[0]	validation_0-mae:0.05896
[786]	validation_0-mae:0.03160
== fold 2 ==
[0]	validation_0-mae:0.06033
[1310]	validation_0-mae:0.03235
== fold 3 ==
[0]	validation_0-mae:0.05630
[1614]	validation_0-mae:0.03187
== fold 4 ==
[0]	validation_0-mae:0.06149
[1337]	validation_0-mae:0.03315


 94%|█████████▍| 17/18 [15:13<00:45, 45.89s/it]

====================z_0====================
== fold 0 ==
[0]	validation_0-mae:0.02615
[401]	validation_0-mae:0.02559
== fold 1 ==
[0]	validation_0-mae:0.02583
[475]	validation_0-mae:0.02514
== fold 2 ==
[0]	validation_0-mae:0.02682
[376]	validation_0-mae:0.02618
== fold 3 ==
[0]	validation_0-mae:0.02685
[683]	validation_0-mae:0.02625
== fold 4 ==
[0]	validation_0-mae:0.02609
[454]	validation_0-mae:0.02551


100%|██████████| 18/18 [15:31<00:00, 51.73s/it]


In [15]:
mae = np.abs(train["oof"] - train["target"]).mean()
print(f"MAE: {mae}")

MAE: 0.21828784498770645


In [16]:
rev_dict = {v: "oof_" + k for k, v in CATEGORY_MAPPING["target_name"].items()}
oof_df = (
    train.select(["ID", "target_name", "oof"])
    .with_columns(pl.col("target_name").replace_strict(rev_dict))
    .pivot(index="ID", columns="target_name", values="oof")
)
oof_df.head()

ID,oof_x_5,oof_y_5,oof_z_5,oof_x_4,oof_y_4,oof_z_4,oof_x_3,oof_y_3,oof_z_3,oof_x_2,oof_y_2,oof_z_2,oof_x_1,oof_y_1,oof_z_1,oof_x_0,oof_y_0,oof_z_0
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""00066be8e20318869c38c66be46663…",21.870636,-0.40243,0.140438,18.210474,-0.282908,0.133933,13.613016,-0.100169,0.067414,9.795164,-0.071299,0.052183,5.879181,-0.033015,0.023066,2.645556,-0.002747,0.016497
"""00066be8e20318869c38c66be46663…",31.578716,-0.569545,0.12726,26.52356,-0.576579,0.152131,21.183722,-0.427999,0.091575,15.8291,-0.386211,0.053785,10.585698,-0.235717,0.036184,5.036321,-0.069186,0.023402
"""00066be8e20318869c38c66be46663…",31.899765,-0.341625,0.100391,26.644663,-0.390511,0.1468,21.071726,-0.178021,0.078273,15.45903,-0.147338,0.039608,10.14452,-0.074818,0.026445,4.71555,-0.023173,0.013426
"""000fb056f97572d384bae4f5fc1e0f…",14.755453,-0.692768,0.056329,11.750469,-0.56795,0.045361,9.07926,-0.342746,0.023428,6.328568,-0.299098,0.016049,3.768057,-0.12705,0.009206,1.601007,-0.049232,0.003031
"""000fb056f97572d384bae4f5fc1e0f…",17.830925,1.074196,-0.000891,14.914308,0.789287,-0.007057,11.813318,0.491941,-0.000635,8.814574,0.264492,0.002926,5.745161,0.107982,0.008025,2.719719,0.020252,0.001921


In [17]:
oof_df.write_csv(os.path.join(CFG["output_dir"], "oof.csv"))